<h3> In this notebook we explore VaR models

<h5> VaR = "How bad can my losses be on a normal bad day?"

- VaR at 95% = the loss you should not exceed on 95% of days

1. Parametric VaR: Assumes that returns are normally distributed
- VaR = mean + vol * normanl quantile

2. Historical Simulation VaR: Assumes that the past = the future
- Just sort past returns and take the worst x%

3. Monte Carlo VaR: Fit a model to returns -> simulate many fake futures -> compute VaR from simulations

4. Expected Shortfall: Focuses on the tail beyond VaR and tells us how bad losses are when VaR is breached. ES at 95% = the average loss in the worst 5% of days.

<h3> Additional Notes

<h5>

- Now, we are calculating a 1-day VaR. For 10-day VaR, need to scale volatility
- More negative VaR = riskier
- To convert log return to percentage loss = 1 - exp^(log return)
- Business interpretation: There is a 5% chance we lose more than {percentage loss} x {portfolio value} tomorrow
- There are different distribution assumptions e.g. Normal (simple, but underestimates tails), Student-t (fat tails which are more realistic), Skewed-t (asymmetry), Empirical (No assumptions)
- There are different risk horizons e.g. 1 day (for trading desks), 10 days (regulatory (basel)), 1-month (asset management)

- VaR: "where is the cutoff"; ES: "if things go badly, how bad do they get"

In [8]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [9]:
returns = pd.read_csv("../data/log_returns.csv", index_col=0, parse_dates=True)
print(returns.head())

            SP500_log  EURUSD_log
DATE                             
2016-01-26   0.014045    0.001292
2016-01-27  -0.010923    0.001934
2016-01-28   0.005513    0.007791
2016-01-29   0.024459   -0.011017
2016-02-01  -0.000443    0.005157


<h3> 0. Choosing weights for portfolio

In [60]:
w = np.array([0.6, 0.4])  # Portfolio weights: 60% SP500, 40% EURUSD
port_returns = returns[['SP500_log', 'EURUSD_log']].dot(w)
port_returns = port_returns.to_frame()
port_returns.columns = ['port_log']
port_returns.to_csv("../data/portfolio_log_returns.csv")
port_returns.head()

,port_log
DATE,
2016-01-26,0.008944
2016-01-27,-0.005780
2016-01-28,0.006425
2016-01-29,0.010268
2016-02-01,0.001797


<h3> 1. Parametric VaR

- Pros: Simple, Fast
- Cons: Bad in crises, Underestimates tail risk

In [40]:
# SP500 Parametric VaR
alpha = 0.05  # 95% VaR
mu = returns['SP500_log'].mean()
sigma = returns['SP500_log'].std()

VaR_param_SP500 = mu + sigma * norm.ppf(alpha)
print(f"Parametric VaR for SP500 at 95% confidence: {VaR_param_SP500}")
print(f"On 95% of days, losses should not exceed {abs(VaR_param_SP500):.4f} in log returns.")

VaR_pct = 1 - np.exp(VaR_param_SP500)
print(f"On 95% of days, losses should not exceed {VaR_pct:.2%}.")

Parametric VaR for SP500 at 95% confidence: -0.018226570668009347
On 95% of days, losses should not exceed 0.0182 in log returns.
On 95% of days, losses should not exceed 1.81%.


In [33]:
# EURUSD Parametric VaR
alpha = 0.05  # 95% VaR
mu = returns['EURUSD_log'].mean()
sigma = returns['EURUSD_log'].std()

VaR_param_EURUSD = mu + sigma * norm.ppf(alpha)
print(f"Parametric VaR for EURUSD at 95% confidence: {VaR_param_EURUSD}")

Parametric VaR for EURUSD at 95% confidence: -0.007576902386920178


In [34]:
# Portfolio Parametric VaR
alpha = 0.05  # 95% VaR
mu = port_returns['port_log'].mean()
sigma = port_returns['port_log'].std()

VaR_param_port = mu + sigma * norm.ppf(alpha)
print(f"Parametric VaR for portfolio at 95% confidence: {VaR_param_port}")

Parametric VaR for portfolio at 95% confidence: -0.01168655591206114


<h3> 2. Historical VaR

- Pros: Very intuitive, Captures fat tails
- Cons: Assumes history repeats, Sensitive to window size

In [35]:
VaR_hist_SP500 = returns['SP500_log'].quantile(0.05)
print(f"Historical VaR for SP500 at 95% confidence: {VaR_hist_SP500}")

VaR_hist_EURUSD = returns['EURUSD_log'].quantile(0.05)
print(f"Historical VaR for EURUSD at 95% confidence: {VaR_hist_EURUSD}") 

VaR_hist_portfolio = port_returns['port_log'].quantile(0.05)
print(f"Historical VaR for portfolio at 95% confidence: {VaR_hist_portfolio}")


Historical VaR for SP500 at 95% confidence: -0.01711285171268744
Historical VaR for EURUSD at 95% confidence: -0.0072513832540862504
Historical VaR for portfolio at 95% confidence: -0.010684768401762208


<h3> 3. Monte Carlo VaR

- Fit a model to returns -> Simulate many fake futures -> Compute VaR from simulations

- Pros: It is flexible and extendable (copulas (models dependence separately, used especially for multi-asset VaR), GARCH (models volatility clusters, used for better VaR during stress), multivariate)
- Cons: Very dependent on data processing

<h3> 4. Expected shortfall

- Use monte carlo to get the mean of the losses below the threshold

In [54]:
# sp500 Monte Carlo Simulated VaR
# 1. Fit distribution
mu = returns['SP500_log'].mean()
sigma = returns['SP500_log'].std()

# 2. Simulate returns
n_sim = 100_000
simulated_returns = np.random.normal(mu, sigma, n_sim)

# 3. Calculate VaR
VaR_simulated = np.percentile(simulated_returns, 5)
print(f"\n Monte Carlo Simulated VaR for SP500 at 95% confidence: {VaR_simulated}")
print(f"\n On 95% of days, losses should not exceed: ~ {VaR_simulated:.4f}")

# 4. Calculate Expected Shortfall (ES)
ES_simulated = simulated_returns[simulated_returns <= VaR_simulated].mean()
print(f"\n Monte Carlo Expected Shortfall for SP500 at 95% confidence: {ES_simulated}")
print(f"\n On the worst 5% of days, the average loss is: ~ {ES_simulated:.4f}")


 Monte Carlo Simulated VaR for SP500 at 95% confidence: -0.01824218638165336

 On 95% of days, losses should not exceed: ~ -0.0182

 Monte Carlo Expected Shortfall for SP500 at 95% confidence: -0.022969165710169933

 On the worst 5% of days, the average loss is: ~ -0.0230


In [42]:
# EURUSD Monte Carlo Simulated VaR
# 1. Fit distribution
mu = returns['EURUSD_log'].mean()
sigma = returns['EURUSD_log'].std()

# 2. Simulate returns
n_sim = 100_000
simulated_returns = np.random.normal(mu, sigma, n_sim)

# 3. Calculate VaR
VaR_simulated = np.percentile(simulated_returns, 5)
print(f"Monte Carlo Simulated VaR for EURUSD at 95% confidence: {VaR_simulated}")

# 4. Calculate Expected Shortfall (ES)
ES_simulated = simulated_returns[simulated_returns <= VaR_simulated].mean()
print(f"Monte Carlo Expected Shortfall for EURUSD at 95% confidence: {ES_simulated}")

Monte Carlo Simulated VaR for EURUSD at 95% confidence: -0.007622839413257749
Monte Carlo Expected Shortfall for EURUSD at 95% confidence: -0.009499146638010028


In [41]:
# Portfolio Monte Carlo Simulated VaR
# 1. Fit distribution
mu = port_returns['port_log'].mean()
sigma = port_returns['port_log'].std()

# 2. Simulate returns
n_sim = 100_000
simulated_returns = np.random.normal(mu, sigma, n_sim)

# 3. Calculate VaR
VaR_simulated = np.percentile(simulated_returns, 5)
print(f"Monte Carlo Simulated VaR for Portfolio at 95% confidence: {VaR_simulated}")

# 4. Calculate Expected Shortfall (ES)
ES_simulated = simulated_returns[simulated_returns <= VaR_simulated].mean()
print(f"Monte Carlo Expected Shortfall for Portfolio at 95% confidence: {ES_simulated}")


Monte Carlo Simulated VaR for Portfolio at 95% confidence: -0.011645104465063722
Monte Carlo Expected Shortfall for Portfolio at 95% confidence: -0.014724257574674791


<h3> Rolling VaR series for backtesting

- backtesting asks: did today's loss exceed today's VaR

In [ ]:
window = 250  # 1 year of trading days
alpha = 0.05  # 95% VaR
n_sim = 100_000

VaR_series = []

for i in range(window, len(port_returns)):
    window_data = port_returns['port_log'].iloc[i-window:i]

    mu = window_data.mean()
    sigma = window_data.std()

    simulated_returns = np.random.normal(mu, sigma, n_sim)
    VaR_t = np.percentile(simulated_returns, alpha * 100)

    VaR_series.append(VaR_t)

VaR_series = pd.Series(VaR_series, index=port_returns.index[window:], name="VaR_MC_95")


In [57]:
# VaR value is the VaR computed using data from the past window
print(f"VaR series head:\n{VaR_series.head()}")

VaR series head:
DATE
2017-01-26   -0.007791
2017-01-27   -0.007850
2017-01-30   -0.007807
2017-01-31   -0.007859
2017-02-01   -0.007840
Name: VaR_MC_95, dtype: float64


In [58]:
# Exporting
VaR_series.to_csv("../data/var_mc_95.csv")